# Programming On Linux




## 1 The software for programming C/C++ and  Python on Linux

### C/C++

* ❑ GCC: The GNU Compiler Collection, containing the GNU C compiler

* ❑ G++: A C++ compiler, included as part of GCC

* ❑ GNU make: A version of UNIX make

Most Linux distributions, such as Ubuntu, have built-in GCC compilers and do not require additional installation


If you use the Debian/Ubuntu distribution do not has built-in GCC compilers. you can install the folloing command
 
```bash 
$sudo apt install build-essential
```

The command installs a bunch of new packages including gcc, g++ and make.

### Python

All Linux distributions have built-in Python2 and Python2. However, it is incomplete

* Add 3 to the relevant commands of Python3


For Python3, we need to be supplemented with `pip3` and `idle3`.

```bash
$sudo apt install python3-pip
$sudo apt install idle3
```

**Install Python3 packages:**


* set `tsinghua` as the mirror site of Pypi

```bash
$sudo -H python3 -m pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
```
* upgrade pip3

```bash
$sudo -H python3 -m  pip install pip -U
```
* Installing Packages

```bash
$sudo -H python3 -m pip install jupyter 
```

```bash
$sudo -H  python3 -m pip install coolprop 
```

### The text editor and programming environment

**Visual Studio Code**

From  https://code.visualstudio.com/docs/?dv=linux64_deb  download VS Code for Debian/Ubuntu,

then install in the terminal

```bash
$sudo dpkg -i [vscode filename].deb
```



### Version Control:Git

```bash 
$sudo apt install git
```


   
## 2 C/C++

In [1]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [1]:
%pwd

'/mnt/j/SEU/SEE/PySEE/home/notebook'

### 2.1 Bisection Methond in C

𝑦=𝑓(𝑥) 

* ./include/roots.h

* ./src/roots.c

In [ ]:
# %load ./include/roots.h
#ifndef ROOTS_H
#define ROOTS_H

#include <math.h>
// function definition
typedef double (*fun)(double);

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb);
/*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
#endif


In [ ]:
# %load ./src/roots.c
/*
  Numerical Recipes http://numerical.recipes
*/ 
#include <math.h>
#include "roots.h"

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)
{
 /*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  If =0, the solution has been computed satisfactorily.
	  If =1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
      If =2, exceeded the maximum number of iteration 
*/
	const int IMAX=100; // the maximum number of iteration
    float dx,f,fmid,xmid;

	f=(*func)(x1)-y;
	fmid=(*func)(x2)-y;
	if (f*fmid >= 0.0) // endpoints do not straddle y=0
       return 1; 
    // init the root value: rtb
	*rtb = f < 0.0 ? (dx=x2-x1,x1) : (dx=x1-x2,x2);
	for (int i=1;i<=IMAX;i++) {
		fmid=(*func)(xmid=(*rtb)+(dx *= 0.5))-y;
		if (fmid <= 0.0) *rtb=xmid;
		if (fabs(dx) < xacc || fmid == 0.0) 
           return 0;
  	}
    // Exceeded the maximum number of iteration
    return 2;
}


### 2.2 The caller of Bisection Methond 

$𝑓(𝑥)=𝑥2$ 

$4=𝑓(𝑥)$

```c
int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)

double xl, xu, epsilon, root;
ier=rtbis(fcn,y,xl, xu, epsilon, &root);
```
&root > double *rtb

In [ ]:
# %load ./src/rtbisApp.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include "roots.h"

double fcn(double x)
{
	double result;
	result = x * x;
	return result;
}

int main()
{
	double xl, xu, epsilon, root;
	int ier;
	xl = 0.1;
	xu = 3.2;
	epsilon = 0.001;
    double y=4;
	// Calculate root
	ier=rtbis(fcn,y,xl, xu, epsilon, &root);
	// Print answers
	printf("root = %14.7e  ier = %1d", root, ier);
	return 0;
}


In [2]:
!gcc -o ./bin/rtbisApp ./src/rtbisApp.c  ./src/roots.c -I./include/

In [3]:
!./bin/rtbisApp

root =  1.9996581e+00  ier = 0

**Further Reading:**

**Path separators**

* Linux: /

* Windows: \

* Eescape character \

The backslash`\` is an escape character,you use to inform that the next character is special.


### 2.3 The shared library

using `__cdecl` calling convention

GCC assumes that all libraries

* start with `lib`

* Shared libraries: end with `.so`(Linux)

#### 2.3.1 creating the shared  library

In [4]:
!gcc -c -O3 -Wall -fPIC -o ./obj/roots.o  ./src/roots.c -I./include
!gcc -shared -o ./bin/libroots.so  ./obj/roots.o

In [5]:
!ls ./bin/lib*.*

./bin/libfuns.dll  ./bin/libroots.so  ./bin/libstat.dll


### 2.3.2 Using the shared library

We use the same code `./src/statdemo.c` under Windows

**Linking the library**

**Windows**
```bash
    -L./bin/ -lstat
```
**Linux**
```bash
   -L./bin/ -lstat -Wl,-rpath=./bin/  
```
* **`-Wl,option`**

    Pass option as an option to the **linker**. 
    
    If option contains `commas`, it is split into multiple options at the commas. 
    
    You can use this syntax to pass an argument to the option.
      
   
* **`-rpath=dir`** 

   **Add a directory to the runtime library search path**. This is used when linking an >executable with shared objects. 
 
All `-rpath` arguments are concatenated and passed to the >runtime linker, which uses them to locate shared objects at runtime. 
   




In [6]:
!gcc -o  ./bin/rtbisApp ./src/rtbisApp.c -L./bin/ -lroots -I./include -Wl,-rpath=./bin/ 

In [7]:
!./bin/rtbisApp

root =  1.9996581e+00  ier = 0

## 3 Using The CoolProp Shared Library

**Change working dir to `./notebook/coolpropdemo`**

In [2]:
%cd coolpropdemo

/mnt/j/SEU/SEE/PySEE/home/notebook/coolpropdemo


In [3]:
%pwd

'/mnt/j/SEU/SEE/PySEE/home/notebook/coolpropdemo'

`libCoolProp.so.6.4.1`  is the library builded with GCC 9.3 and glibc2.29 above on Linux

type the following command in the terminal of `./coolpropdemo/` to create the symbolic link `libCoolProp.so`

```bash
$sudo ln -sf libCoolProp.so.6.4.1 ./bin/libCoolProp.so.6
$sudo ln -sf libCoolProp.so.6 ./bin/libCoolProp.so
```

In [12]:
%%file ./mainso.c

/* 
The example use the shared library of CoolProp in c

g++ -o ./bin/mainso  -DCOOLPROP_LIB mainso.c -I./include -L./bin -lCoolProp -Wl,-rpath=./bin/ 
       
./bin/mainso

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}

Overwriting ./mainso.c


In [10]:
!g++ -o ./bin/mainso  -DCOOLPROP_LIB mainso.c -I./include -L./bin -lCoolProp -Wl,-rpath=./bin/ 

In [11]:
!./bin/mainso

292803.182339


## Reference

https://github.com/PySEE/home/blob/B2021/guide/doc/Ubuntu-Python-CPP(Chinese).md